In [1]:
from bertopic import BERTopic

W0819 19:56:03.415000 36696 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Traceback (most recent call last):
  File "C:\Users\giovannt\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model = BERTopic.load("/data/bertopic_manual", embedding_model=embedding_model)

In [ ]:
topic_model.visualize_topics(top_n_topics = 18)

In [ ]:
# Visualize the topics
topic_model.visualize_barchart(top_n_topics=60).show()

In [ ]:
topic_info = topic_model.get_topic_info()

In [ ]:
# Print the most relevant topics
print(topic_info.head(16))

In [ ]:
df_chilit = pd.read_csv("./data/ChiLit_Chunks_200.csv")
df_chilit = df_chilit.fillna("")
sentences = df_chilit['tokens'].to_list()

In [ ]:
book_ids = df_chilit['book_id'].to_list()
probs_assigned = [p[t] if t != -1 else 0.0 for p, t in zip(probs, topics)]
df = pd.DataFrame({
    "doc_id": book_ids,
    "chunk": sentences,
    "topic": topics,
    "prob" : probs_assigned
})

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(sentences)


# Run the visualization with the original embeddings
topic_model.visualize_hierarchical_documents(sentences, hierarchical_topics, embeddings=embeddings)


#topic_model.visualize_hierarchical_documents(sentences, hierarchical_topics, reduced_embeddings=reduced_embeddings)

In [ ]:
# Reduce dimensionality of embeddings, this step is optional
#reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

# Or, if you have reduced the original embeddings already:
topic_model.visualize_hierarchical_documents(sentences, hierarchical_topics, reduced_embeddings=reduced_embeddings)

In [ ]:
# 1. Aggregate total probability per (doc_id, topic)
topic_confidence = (
    df[df.topic != -1]
    .groupby(["doc_id", "topic"])["prob"]
    .sum()
    .reset_index()
)

# 2. Pivot into matrix form (rows = doc_id, cols = topic)
pivot_df = topic_confidence.pivot(
    index="doc_id", columns="topic", values="prob"
).fillna(0)

# 3. Plot heatmap
plt.figure(figsize=(8, 16))
sns.heatmap(
    pivot_df,
    annot=False, fmt=".2f", cmap="YlGnBu",
    cbar_kws={"label": "Total topic confidence"}
)
plt.title("Document vs Topic Confidence (Option B Aggregation)")
plt.xlabel("Topic ID")
plt.ylabel("Original Document ID")
plt.tight_layout()

plt.show()

# 4. Get best topic for each document
best_topics = pivot_df.idxmax(axis=1)
print("\nBest topic per document:")
print(best_topics)

In [ ]:
def evaluate_model(df_chilit, topic_model, sentences):

  book_ids = df_chilit['book_id'].to_list()

  df_document_info = topic_model.get_document_info(sentences)
  topics = df_document_info['Topic'].to_list()
  probs = df_document_info['Probability'].to_list()
  chunks = df_document_info['Document'].to_list()

  df = pd.DataFrame({
      "doc_id": book_ids,
      "chunk": sentences,
      "topic": topics,
      "prob" : probs
  })

  # 1. Aggregate total probability per (doc_id, topic)
  topic_confidence = (
      df[df.topic != -1]
      .groupby(["doc_id", "topic"])["prob"]
      .mean()
      .reset_index()
  )

  # 2. Pivot into matrix form (rows = doc_id, cols = topic)
  pivot_df = topic_confidence.pivot(
      index="doc_id", columns="topic", values="prob"
  ).fillna(0)

  # 3. Plot heatmap
  plt.figure(figsize=(10, 15))
  sns.heatmap(
      pivot_df,
      annot=True, fmt=".2f", cmap="YlGnBu",
      cbar_kws={"label": "Total topic confidence"}
  )
  plt.title("Document vs Topic Confidence")
  plt.xlabel("Topic ID")
  plt.ylabel("Original Document ID")
  plt.tight_layout()

  plt.show()

  # 4. Get best topic for each document
  best_topics = pivot_df.idxmax(axis=1)
  print("\nBest topic per document:")
  print(best_topics)

evaluate_model(df_chilit, final_model, sentences)